In [1]:
# Import pandas for data manipulation and SQLAlchemy for database interaction

import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Define the path to the CSV file and the SQLite database file

csv_path = '/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/liverpool_crime_data.csv'

db_path = '/Users/jamesjackson/Documents/liverpool_crime_analysis/liverpool_crime.db'

In [4]:
# Read the CSV into a pandas DataFrame

df = pd.read_csv(csv_path)

# Check the first few rows to confirm it loaded correctly
df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution,NaN
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified,NaN
2,NaN,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,NaN,NaN
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified,NaN
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect,NaN


In [5]:
# Create a connection engine to the SQLite database file

engine = create_engine(f'sqlite:///{db_path}')

In [6]:
# Write the DataFrame to a table named 'crime_data' in the database
# If the table exists, replace it

df.to_sql('crime_data', engine, if_exists='replace', index=False)


150535

In [8]:
from sqlalchemy import text

# Check what tables exist in the database now

with engine.connect() as conn:

    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))

    tables = [row[0] for row in result]

print("Tables in database:", tables)


Tables in database: ['crime_data']


In [10]:
from sqlalchemy import text

# Run a SQL query to count total rows in 'crime_data'

with engine.connect() as conn:

    result = conn.execute(text("SELECT COUNT(*) AS total_rows FROM crime_data;"))

    total_rows = result.fetchone()[0]

print("Total rows in crime_data:", total_rows)

Total rows in crime_data: 150535


In [12]:
from sqlalchemy import text

# Step 8: Count distinct crime types in the data

with engine.connect() as conn:
    result = conn.execute(text('SELECT COUNT(DISTINCT "Crime type") AS unique_crime_types FROM crime_data;'))
    unique_crimes = result.fetchone()[0]

print("Unique crime types:", unique_crimes)

Unique crime types: 14


In [14]:
# Example: Get all records for a specific crime type, e.g. 'Burglary'

query = """
SELECT *
FROM crime_data
WHERE "Crime type" = 'Burglary'
LIMIT 5;
"""

with engine.connect() as conn:
    burglary_samples = pd.read_sql_query(query, conn)

burglary_samples

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,299a8fa1ce72c3e87de73560bee9faf6606862f1a06079...,2024-06,Merseyside Police,Merseyside Police,-2.867785,53.490368,On or near Warrenhouse Road,E01006494,Knowsley 001D,Burglary,Investigation complete; no suspect identified,None
1,3f68851792c5677094ba81c48a49e848b98fd93f76af3d...,2024-06,Merseyside Police,Merseyside Police,-2.878243,53.496134,On or near Stocks Road,E01006494,Knowsley 001D,Burglary,Court result unavailable,None
2,d33996c2cf26ee35ce4eabcc681cde10ba4c8cde77bfbe...,2024-06,Merseyside Police,Merseyside Police,-2.876911,53.465888,On or near Old Farm Road,E01006418,Knowsley 005B,Burglary,Investigation complete; no suspect identified,None
3,68bd39d6a9d022fb342e3d319439d4752f3af80b5db69b...,2024-06,Merseyside Police,Merseyside Police,-2.860352,53.479851,On or near Webber Road,E01006446,Knowsley 005G,Burglary,Investigation complete; no suspect identified,None
4,4a7bc9cb21d15e0b628d789282b159bc02e8d62e708c28...,2024-06,Merseyside Police,Merseyside Police,-2.863481,53.438022,On or near Blackthorne Crescent,E01006413,Knowsley 006B,Burglary,Investigation complete; no suspect identified,None


In [21]:
from sqlalchemy import text

# Count how many rows have a missing Crime ID (NULL values)
with engine.connect() as conn:
    result = conn.execute(text('SELECT COUNT(*) FROM crime_data WHERE "Crime ID" IS NULL;'))
    missing_crime_id_count = result.scalar()

print(f"Number of rows with missing Crime ID: {missing_crime_id_count}")

Number of rows with missing Crime ID: 16252


In [22]:
# Check for duplicated Crime IDs by counting how many have more than one occurrence

from sqlalchemy import text

with engine.connect() as conn:
    query = text("""
        SELECT "Crime ID", COUNT(*) AS count
        FROM crime_data
        WHERE "Crime ID" IS NOT NULL
        GROUP BY "Crime ID"
        HAVING COUNT(*) > 1;
    """)
    result = conn.execute(query)
    duplicates = result.fetchall()

print(f"Number of duplicated Crime IDs: {len(duplicates)}")


Number of duplicated Crime IDs: 0


In [31]:
import pandas as pd
from sqlalchemy import text

# Fetch rows with NULL Crime ID and show as DataFrame
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM crime_data WHERE "Crime ID" IS NULL LIMIT 15;'))
    null_entries = result.fetchall()
    columns = result.keys()

null_df = pd.DataFrame(null_entries, columns=columns)
null_df.head(15)



,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,None,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,None,None
1,None,2024-06,Merseyside Police,Merseyside Police,-2.870433,53.490888,On or near Woodcote Close,E01006494,Knowsley 001D,Anti-social behaviour,None,None
2,None,2024-06,Merseyside Police,Merseyside Police,-2.878943,53.489891,On or near Roughwood Drive,E01006494,Knowsley 001D,Anti-social behaviour,None,None
3,None,2024-06,Merseyside Police,Merseyside Police,-2.878943,53.489891,On or near Roughwood Drive,E01006494,Knowsley 001D,Anti-social behaviour,None,None
4,None,2024-06,Merseyside Police,Merseyside Police,-2.879075,53.491876,On or near Highbarn Road,E01006495,Knowsley 001E,Anti-social behaviour,None,None
5,None,2024-06,Merseyside Police,Merseyside Police,-2.882803,53.489970,On or near Brookwood Court,E01006495,Knowsley 001E,Anti-social behaviour,None,None
6,None,2024-06,Merseyside Police,Merseyside Police,-2.899482,53.489684,On or near Chiltern Drive,E01006454,Knowsley 002A,Anti-social behaviour,None,None
7,None,2024-06,Merseyside Police,Merseyside Police,-2.904851,53.491243,On or near Beldale Park,E01006454,Knowsley 002A,Anti-social behaviour,None,None
8,None,2024-06,Merseyside Police,Merseyside Police,-2.887114,53.484221,On or near Hall Drive,E01006457,Knowsley 002C,Anti-social behaviour,None,None
9,None,2024-06,Merseyside Police,Merseyside Police,-2.887114,53.484221,On or near Hall Drive,E01006457,Knowsley 002C,Anti-social behaviour,None,None


In [32]:
from sqlalchemy import text

# Count how many rows with NULL Crime ID have Crime type 'Anti-social behaviour'
with engine.connect() as conn:
    result = conn.execute(text('''
        SELECT COUNT(*) 
        FROM crime_data 
        WHERE "Crime ID" IS NULL AND "Crime type" = 'Anti-social behaviour';
    '''))
    count = result.scalar()

print(f"Number of NULL Crime ID rows with 'Anti-social behaviour': {count}")



Number of NULL Crime ID rows with 'Anti-social behaviour': 16252


In [34]:
from sqlalchemy import text

# Count how many rows with non-NULL Crime ID have Crime type 'Anti-social behaviour'
with engine.connect() as conn:
    result = conn.execute(text('''
        SELECT COUNT(*) 
        FROM crime_data 
        WHERE "Crime ID" IS NOT NULL AND "Crime type" = 'Anti-social behaviour';
    '''))
    count_non_null = result.scalar()

print(f"Number of non-NULL Crime ID rows with 'Anti-social behaviour': {count_non_null}")

#Learnt that anti-social behaviour crimes do not have an associated Crime ID

Number of non-NULL Crime ID rows with 'Anti-social behaviour': 0


In [37]:
from sqlalchemy import text

# Update Crime ID to 'ASB_FILL' where Crime ID is NULL and Crime type is 'Anti-social behaviour'
with engine.connect() as conn:
    conn.execute(
        text("""
            UPDATE crime_data
            SET "Crime ID" = 'ASB_FILL'
            WHERE "Crime ID" IS NULL
            AND "Crime type" = 'Anti-social behaviour';
        """)
    )
    conn.commit()  # Commit the transaction


In [40]:
from sqlalchemy import text
import pandas as pd

# Use engine.begin() for automatic commit
with engine.begin() as conn:
    conn.execute(
        text("""
            UPDATE crime_data
            SET "Crime ID" = 'ASB_FILL'
            WHERE "Crime ID" IS NULL
            AND "Crime type" = 'Anti-social behaviour';
        """)
    )

    # Now fetch updated rows to confirm
    result = conn.execute(
        text("SELECT * FROM crime_data WHERE \"Crime ID\" = 'ASB_FILL' LIMIT 5;")
    )
    updated_rows = result.fetchall()
    columns = result.keys()

updated_df = pd.DataFrame(updated_rows, columns=columns)
updated_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,None,None
1,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.870433,53.490888,On or near Woodcote Close,E01006494,Knowsley 001D,Anti-social behaviour,None,None
2,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.878943,53.489891,On or near Roughwood Drive,E01006494,Knowsley 001D,Anti-social behaviour,None,None
3,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.878943,53.489891,On or near Roughwood Drive,E01006494,Knowsley 001D,Anti-social behaviour,None,None
4,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.879075,53.491876,On or near Highbarn Road,E01006495,Knowsley 001E,Anti-social behaviour,None,None


In [42]:
# Reload fresh data from the database after update
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM crime_data LIMIT 5;'))
    rows = result.fetchall()
    columns = result.keys()

fresh_df = pd.DataFrame(rows, columns=columns)
fresh_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution,None
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified,None
2,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,None,None
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified,None
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect,None


In [43]:
from sqlalchemy import text

# List columns to check for missing values (NULL or empty string)
columns_to_check = [
    "Crime ID", "Month", "Reported by", "Falls within", "Longitude",
    "Latitude", "Location", "LSOA code", "LSOA name", "Crime type",
    "Last outcome category", "Context"
]

with engine.connect() as conn:
    for col in columns_to_check:
        # SQL query to count rows where column is NULL or empty string
        query = text(f'SELECT COUNT(*) AS missing_count FROM crime_data WHERE "{col}" IS NULL OR "{col}" = \'\';')
        result = conn.execute(query)
        missing_count = result.scalar()
        print(f"Missing in {col}: {missing_count}")  # Display missing counts per column


Missing in Crime ID: 0
Missing in Month: 0
Missing in Reported by: 0
Missing in Falls within: 0
Missing in Longitude: 0
Missing in Latitude: 0
Missing in Location: 0
Missing in LSOA code: 0
Missing in LSOA name: 0
Missing in Crime type: 0
Missing in Last outcome category: 16252
Missing in Context: 150535


In [45]:
# Drop the 'Context' column because it is completely missing (all values are NULL)
with engine.connect() as conn:
    conn.execute(text('ALTER TABLE crime_data RENAME TO crime_data_old;'))
    conn.execute(text('''
        CREATE TABLE crime_data AS
        SELECT 
            "Crime ID", "Month", "Reported by", "Falls within",
            "Longitude", "Latitude", "Location", "LSOA code",
            "LSOA name", "Crime type", "Last outcome category"
        FROM crime_data_old;
    '''))
    conn.execute(text('DROP TABLE crime_data_old;'))
    conn.commit()

# Confirm the column is dropped by checking columns
with engine.connect() as conn:
    result = conn.execute(text("PRAGMA table_info(crime_data);"))
    columns = [row[1] for row in result.fetchall()]

import pandas as pd
pd.DataFrame(columns, columns=['Columns'])


,Columns
0,Crime ID
1,Month
2,Reported by
3,Falls within
4,Longitude
5,Latitude
6,Location
7,LSOA code
8,LSOA name
9,Crime type


In [46]:
# Fetch the first 10 rows from the updated crime_data table to check the current state of the data
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM crime_data LIMIT 10;'))
    rows = result.fetchall()
    columns = result.keys()

import pandas as pd
head_df = pd.DataFrame(rows, columns=columns)
head_df

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified
2,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,None
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect
5,0e5d5be47f6fbb299f9ed00b4aa64cf7984475a98ddfe0...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Drugs,Unable to prosecute suspect
6,e822728454620d8fd43110852d8a16d4482849f60977f0...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Drugs,Court result unavailable
7,88d80f378f168ca985324ef2ebb4463286745500d47038...,2024-06,Merseyside Police,Merseyside Police,-2.870117,53.489452,On or near Jarrett Road,E01006448,Knowsley 001A,Public order,Investigation complete; no suspect identified
8,7bfc4eb5156b7f044982f2ab6f8d82eea5060636c55471...,2024-06,Merseyside Police,Merseyside Police,-2.870190,53.485658,On or near Darmond Road,E01006448,Knowsley 001A,Vehicle crime,Investigation complete; no suspect identified
9,b6fba1fbba7cd0613983f452b27ad3fb62068f2a22b7e7...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Violence and sexual offences,Unable to prosecute suspect


In [49]:
from sqlalchemy import text
import pandas as pd

# As all null values are from ASB crime types, we fill NULL values in 'Last outcome category' with 'ASB_FILL' for 'Anti-social behaviour' crimes

with engine.connect() as conn:
    conn.execute(
        text("""
            UPDATE crime_data
            SET "Last outcome category" = 'ASB_FILL'
            WHERE "Last outcome category" IS NULL
            AND "Crime type" = 'Anti-social behaviour';
        """)
    )
    conn.commit()

    # Fetch first 5 rows of the updated table to confirm
    result = conn.execute(
        text("SELECT * FROM crime_data LIMIT 5;")
    )
    updated_rows = result.fetchall()
    columns = result.keys()

updated_df = pd.DataFrame(updated_rows, columns=columns)
updated_df.head()



,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified
2,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,ASB_FILL
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect


In [50]:
# Rename columns to be lowercase and replace spaces with underscores for consistency

with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE crime_data RENAME TO crime_data_old;
    """))
    # Create new table with renamed columns
    conn.execute(text("""
        CREATE TABLE crime_data (
            crime_id TEXT,
            month TEXT,
            reported_by TEXT,
            falls_within TEXT,
            longitude REAL,
            latitude REAL,
            location TEXT,
            lsoa_code TEXT,
            lsoa_name TEXT,
            crime_type TEXT,
            last_outcome_category TEXT
        );
    """))
    # Copy data from old table to new table, mapping old columns to new names
    conn.execute(text("""
        INSERT INTO crime_data (
            crime_id, month, reported_by, falls_within,
            longitude, latitude, location, lsoa_code,
            lsoa_name, crime_type, last_outcome_category
        )
        SELECT
            "Crime ID", "Month", "Reported by", "Falls within",
            Longitude, Latitude, Location, "LSOA code",
            "LSOA name", "Crime type", "Last outcome category"
        FROM crime_data_old;
    """))
    # Drop old table
    conn.execute(text("DROP TABLE crime_data_old;"))
    conn.commit()

# Fetch first 5 rows to confirm changes and display as DataFrame
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM crime_data LIMIT 5;"))
    rows = result.fetchall()
    columns = result.keys()

renamed_df = pd.DataFrame(rows, columns=columns)
renamed_df.head()


,crime_id,month,reported_by,falls_within,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified
2,ASB_FILL,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,ASB_FILL
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect


In [52]:
# Check the data type of the 'month' column in the DataFrame
print(df['Month'].dtype)

object


In [55]:
import pandas as pd
from sqlalchemy import text

# Fetch a sample of rows from the database
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM crime_data LIMIT 10;'))
    rows = result.fetchall()
    columns = result.keys()

# Create DataFrame from fetched data
df_sample = pd.DataFrame(rows, columns=columns)

# Convert 'month' column from string to datetime (year-month only)
df_sample['month'] = pd.to_datetime(df_sample['month'], format='%Y-%m')

# Show the updated dataframe with datetime 'month' column
df_sample.head(10)


,crime_id,month,reported_by,falls_within,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06-01,Merseyside Police,Merseyside Police,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06-01,Merseyside Police,Merseyside Police,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified
2,ASB_FILL,2024-06-01,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,ASB_FILL
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06-01,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06-01,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect
5,0e5d5be47f6fbb299f9ed00b4aa64cf7984475a98ddfe0...,2024-06-01,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Drugs,Unable to prosecute suspect
6,e822728454620d8fd43110852d8a16d4482849f60977f0...,2024-06-01,Merseyside Police,Merseyside Police,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Drugs,Court result unavailable
7,88d80f378f168ca985324ef2ebb4463286745500d47038...,2024-06-01,Merseyside Police,Merseyside Police,-2.870117,53.489452,On or near Jarrett Road,E01006448,Knowsley 001A,Public order,Investigation complete; no suspect identified
8,7bfc4eb5156b7f044982f2ab6f8d82eea5060636c55471...,2024-06-01,Merseyside Police,Merseyside Police,-2.870190,53.485658,On or near Darmond Road,E01006448,Knowsley 001A,Vehicle crime,Investigation complete; no suspect identified
9,b6fba1fbba7cd0613983f452b27ad3fb62068f2a22b7e7...,2024-06-01,Merseyside Police,Merseyside Police,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Violence and sexual offences,Unable to prosecute suspect


In [57]:
with engine.connect() as conn:
    result = conn.execute(text('SELECT month FROM crime_data LIMIT 5;'))
    rows = result.fetchall()
    columns = result.keys()

df_check = pd.DataFrame(rows, columns=columns)
print(df_check.dtypes)
df_check.head()


month    object
dtype: object


,month
0,2024-06
1,2024-06
2,2024-06
3,2024-06
4,2024-06


In [59]:
from sqlalchemy import text
import pandas as pd

# Cleaning step: Identify and remove leading/trailing whitespace
# from text-based columns to ensure consistent data formatting
# This helps avoid issues with string comparisons and grouping later.

# Columns to check and trim whitespace from
text_columns = ['reported_by', 'falls_within', 'location', 'lsoa_code', 'lsoa_name', 'crime_type', 'last_outcome_category']

with engine.connect() as conn:
    # Step 1: Check how many rows in each column have leading/trailing whitespace
    for col in text_columns:
        query = text(f'''
            SELECT COUNT(*) AS count_with_whitespace
            FROM crime_data
            WHERE {col} <> TRIM({col});
        ''')
        result = conn.execute(query)
        count = result.scalar()
        print(f"Rows with leading/trailing whitespace in '{col}': {count}")

Rows with leading/trailing whitespace in 'reported_by': 0
Rows with leading/trailing whitespace in 'falls_within': 0
Rows with leading/trailing whitespace in 'location': 0
Rows with leading/trailing whitespace in 'lsoa_code': 0
Rows with leading/trailing whitespace in 'lsoa_name': 0
Rows with leading/trailing whitespace in 'crime_type': 0
Rows with leading/trailing whitespace in 'last_outcome_category': 0


In [69]:
from sqlalchemy import text
import pandas as pd

# Checking distinct values in key categorical columns to identify inconsistencies or cleanup needs

with engine.connect() as conn:
    # Distinct Crime Types
    result = conn.execute(text('SELECT DISTINCT crime_type FROM crime_data;'))
    crime_types = sorted(row[0] for row in result.fetchall())
    print("Crime Types:\n- " + "\n- ".join(crime_types) + "\n")

    # Distinct Reported By
    result = conn.execute(text('SELECT DISTINCT reported_by FROM crime_data;'))
    reported_by = sorted(row[0] for row in result.fetchall())
    print("Reported By:\n- " + "\n- ".join(reported_by) + "\n")

    # Distinct Last Outcome Categories
    result = conn.execute(text('SELECT DISTINCT last_outcome_category FROM crime_data;'))
    last_outcomes = sorted(str(row[0]) for row in result.fetchall())  # convert None to 'None' string if any
    print("Last Outcome Categories:\n- " + "\n- ".join(last_outcomes) + "\n")



Crime Types:
- Anti-social behaviour
- Bicycle theft
- Burglary
- Criminal damage and arson
- Drugs
- Other crime
- Other theft
- Possession of weapons
- Public order
- Robbery
- Shoplifting
- Theft from the person
- Vehicle crime
- Violence and sexual offences

Reported By:
- Merseyside Police

Last Outcome Categories:
- ASB_FILL
- Action to be taken by another organisation
- Awaiting court outcome
- Court result unavailable
- Formal action is not in the public interest
- Further action is not in the public interest
- Further investigation is not in the public interest
- Investigation complete; no suspect identified
- Local resolution
- Offender given a caution
- Offender given a drugs possession warning
- Status update unavailable
- Suspect charged as part of another case
- Unable to prosecute suspect
- Under investigation



In [70]:
# Check distinct values in 'falls_within' to decide if it can be dropped
with engine.connect() as conn:
    result = conn.execute(text('SELECT DISTINCT falls_within FROM crime_data;'))
    distinct_falls_within = [row[0] for row in result.fetchall()]

print("Distinct values in 'falls_within':")
for val in distinct_falls_within:
    print(f"- {val}")


Distinct values in 'falls_within':
- Merseyside Police


In [71]:
# Drop 'reported_by' and 'falls_within' columns as they contain the same repeated values (likely not informative)
with engine.connect() as conn:
    conn.execute(text('ALTER TABLE crime_data DROP COLUMN reported_by;'))
    conn.execute(text('ALTER TABLE crime_data DROP COLUMN falls_within;'))
    conn.commit()

# Fetch a sample to confirm columns dropped
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM crime_data LIMIT 5;'))
    rows = result.fetchall()
    columns = result.keys()

df_head = pd.DataFrame(rows, columns=columns)
df_head.head()


,crime_id,month,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category
0,7c903b55af24fe6dab7eb96fe696f8e7d57e05817f6730...,2024-06,-2.746819,53.389101,On or near Further/Higher Educational Building,E01012393,Halton 001B,Drugs,Local resolution
1,461fe1ff825ef4cc740dafe58e9015449696d80978580a...,2024-06,-2.798814,53.354705,On or near Old Higher Road,E01012391,Halton 008B,Criminal damage and arson,Investigation complete; no suspect identified
2,ASB_FILL,2024-06,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Anti-social behaviour,ASB_FILL
3,ebeec31356de0e9219711d81f3d8006d33c275e04e9e7a...,2024-06,-2.871229,53.489300,On or near Watts Close,E01006448,Knowsley 001A,Criminal damage and arson,Investigation complete; no suspect identified
4,b04da555d9c4f211ce48bf6453dc0f6257c63785727ec6...,2024-06,-2.871827,53.489763,On or near Gilescroft Avenue,E01006448,Knowsley 001A,Criminal damage and arson,Unable to prosecute suspect


In [73]:
from sqlalchemy import text
import pandas as pd

# Check for any rows where 'month' (YYYY-MM) is in the future compared to current year-month
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT month
        FROM crime_data
        WHERE substr(month, 1, 7) > strftime('%Y-%m', 'now')
        LIMIT 5;
    """))
    future_months = result.fetchall()
    columns = result.keys()

future_df = pd.DataFrame(future_months, columns=columns)
future_df

,month


In [74]:
# Save the cleaned dataframe to the specified folder with the filename clean_data.csv
df.to_csv('/Users/jamesjackson/Documents/liverpool_crime_analysis/csv_files/clean_data.csv', index=False)
